<a href="https://colab.research.google.com/github/emmenets/neuralnetwork/blob/master/chinook190715.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
!wget https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [5]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
mkdir images

In [0]:
mkdir inference_graph

In [0]:
mkdir training

In [10]:
ls

anchor_generators/                   inputs_test.py
box_coders/                          legacy/
builders/                            matchers/
CONTRIBUTING.md                      meta_architectures/
core/                                metrics/
data/                                model_hparams.py
data_decoders/                       model_lib.py
dataset_tools/                       model_lib_test.py
dockerfiles/                         model_main.py
eval_util.py                         models/
eval_util_test.py                    model_tpu_main.py
exporter.py                          object_detection_tutorial.ipynb
exporter_test.py                     predictors/
export_inference_graph.py            protos/
export_tflite_ssd_graph_lib.py       README.md
export_tflite_ssd_graph_lib_test.py  samples/
export_tflite_ssd_graph.py           test_ckpt/
g3doc/                               test_data/
images/                              test_images/
inference/                           tpu_

In [13]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/xml_to_csv.py /content/models/research/object_detection

In [38]:
!python3 xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


In [45]:
!cat /content/models/research/object_detection/generate_tfrecord.py

"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('image_dir', '', 'Path to the image directory')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'chinook':
        return 1
    else:
        return 0

In [46]:
%%writefile generate_tfrecord_01.py

"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('image_dir', '', 'Path to the image directory')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'chinook':
        return 1
    else:
        return 0


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

Writing generate_tfrecord_01.py


In [0]:
!python generate_tfrecord_01.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

In [51]:
!python generate_tfrecord_01.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

W0715 10:20:28.904641 140045158999936 deprecation_wrapper.py:119] From generate_tfrecord_01.py:101: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0715 10:20:28.905257 140045158999936 deprecation_wrapper.py:119] From generate_tfrecord_01.py:87: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0715 10:20:28.944823 140045158999936 deprecation_wrapper.py:119] From generate_tfrecord_01.py:46: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/models/research/object_detection/test.record


In [0]:
cp /content/models/research/object_detection/data/pet_label_map.pbtxt /content/models/research/object_detection/training

In [54]:
!cat /content/models/research/object_detection/training/pet_label_map.pbtxt

item {
  id: 1
  name: 'Abyssinian'
}

item {
  id: 2
  name: 'american_bulldog'
}

item {
  id: 3
  name: 'american_pit_bull_terrier'
}

item {
  id: 4
  name: 'basset_hound'
}

item {
  id: 5
  name: 'beagle'
}

item {
  id: 6
  name: 'Bengal'
}

item {
  id: 7
  name: 'Birman'
}

item {
  id: 8
  name: 'Bombay'
}

item {
  id: 9
  name: 'boxer'
}

item {
  id: 10
  name: 'British_Shorthair'
}

item {
  id: 11
  name: 'chihuahua'
}

item {
  id: 12
  name: 'Egyptian_Mau'
}

item {
  id: 13
  name: 'english_cocker_spaniel'
}

item {
  id: 14
  name: 'english_setter'
}

item {
  id: 15
  name: 'german_shorthaired'
}

item {
  id: 16
  name: 'great_pyrenees'
}

item {
  id: 17
  name: 'havanese'
}

item {
  id: 18
  name: 'japanese_chin'
}

item {
  id: 19
  name: 'keeshond'
}

item {
  id: 20
  name: 'leonberger'
}

item {
  id: 21
  name: 'Maine_Coon'
}

item {
  id: 22
  name: 'miniature_pinscher'
}

item {
  id: 23
  name: 'newfoundland'
}

item {
  id: 24
  name: 'Persian'
}

item 